# PyTorch Transfer Learning

#### **1. What is transfer learning?**
So far we have built many models from scratch. However, this is not always the best approach. As we also expereinced our model performed poorly. In many cases, we can use a pre-trained model and fine-tune it to our specific problem. This is called transfer learning.<br>
With the application of **Transfer Learning** we can take weights another model has learned from some other probelem and use them to solve our problem. This is especially useful when we have a small dataset. <br>
In this notebook, we will use a pre-trained model called **ResNet18** and fine-tune it to our pizza-steak-sushi problem. <br>

#### **2. Why Transfer Learning?**
- **Less Data**: Deep learning models require a lot of data. However, in practice, we often have a small dataset. Transfer learning helps us to use pre-trained models and fine-tune them to our specific problem.
- **Computation**: Training a deep learning model from scratch requires a lot of computation. However, with transfer learning, we can use pre-trained models and fine-tune them to our specific problem. This requires less computation.
- **Time**: Training a deep learning model from scratch requires a lot of time. However, with transfer learning, we can use pre-trained models and fine-tune them to our specific problem. This requires less time. <br>
Also, A finding from a recent machine learning research paper recommended practitioners use transfer learning wherever possible.


![Transfer Learning Image](https://www.techtarget.com/rms/onlineimages/how_transfer_learning_works-f.png)
